<a href="https://colab.research.google.com/github/MarceloClaro/MarceloClaro-COMPARA-O_DE_AUT-MATOS_CELULARES_CL-SSICOS_E_QUANTICOS/blob/main/COMPARA%C3%87%C3%83O_DE_AUT%C3%94MATOS_CELULARES_CL%C3%81SSICOS_E_QUANTICOS_Performance%2C_robustez_ao_ru%C3%ADdo_e_aplica%C3%A7%C3%B5es_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import subprocess

# Executar o comando 'nvidia-smi' para obter informações sobre a GPU
gpu_info = subprocess.run(['nvidia-smi'], capture_output=True, text=True).stdout

# Verificar se há falha na conexão com a GPU
if 'failed' in gpu_info.lower():
    print('Não está conectado a uma GPU')
else:
    print(gpu_info)


In [ ]:
from psutil import virtual_memory

# Obter a quantidade total de RAM disponível
ram_gb = virtual_memory().total / 1e9

# Exibir a quantidade de RAM disponível
print(f'Seu tempo de execução tem {ram_gb:.1f} gigabytes de RAM disponível\n')

# Verificar se está usando um tempo de execução com alta RAM
print('Você está usando um tempo de execução com alta RAM!' if ram_gb >= 20 else 'Não está usando um tempo de execução com alta RAM')


In [ ]:
import numpy as np  # Biblioteca para operações com arrays numéricos
import matplotlib.pyplot as plt  # Biblioteca para criar gráficos
import seaborn as sns  # Biblioteca para visualização de dados, baseada no matplotlib
from qiskit import QuantumCircuit, transpile  # Importa o QuantumCircuit para criar circuitos quânticos e transpile para otimização de circuitos
from qiskit.visualization import plot_histogram, plot_bloch_multivector, plot_bloch_vector, circuit_drawer  # Funções de visualização do Qiskit
from qiskit_aer import AerSimulator  # Simulador de aer do Qiskit para simular circuitos quânticos
from qiskit.quantum_info import Statevector, DensityMatrix  # Função para obter o vetor de estado de um circuito quântico
import pandas as pd  # Biblioteca para manipulação e análise de dados
from scipy.stats import entropy  # Função para calcular entropia
import time  # Biblioteca para medir o tempo de execução
import os  # Biblioteca para operações com o sistema operacional

# Função para criar visualização da esfera de Bloch para um vetor de estado
def plot_bloch_sphere(state_vector):
    bloch_vector = Statevector(state_vector).data.real  # Obtém o vetor de Bloch
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plotar a esfera de Bloch
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 100)
    x = np.outer(np.cos(u), np.sin(v))
    y = np.outer(np.sin(u), np.sin(v))
    z = np.outer(np.ones(np.size(u)), np.cos(v))
    ax.plot_surface(x, y, z, color='b', alpha=0.1)

    # Plotar o vetor de Bloch
    ax.quiver(0, 0, 0, bloch_vector[0], bloch_vector[1], bloch_vector[2], color='r', arrow_length_ratio=0.3)

    ax.set_xlim([-1, 1])
    ax.set_ylim([-1, 1])
    ax.set_zlim([-1, 1])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()

# Função para converter DataFrame clássico em operações quânticas e executar a simulação
def run_quantum_simulation_from_classical(df_classical, block_size, rule_number):
    simulator = AerSimulator()  # Inicializa o simulador de Aer
    results = []  # Lista para armazenar os resultados
    num_blocks = (len(df_classical.columns) + block_size - 1) // block_size  # Calcula o número de blocos
    circuit_count = 1  # Contador de circuitos

    for index, row in df_classical.iterrows():  # Itera sobre cada linha do DataFrame
        block_results = []
        for block in range(num_blocks):  # Itera sobre cada bloco
            start = block * block_size  # Início do bloco
            end = min(start + block_size, len(df_classical.columns))  # Fim do bloco
            block_state = row.values[start:end]  # Obtém o estado do bloco
            qc = QuantumCircuit(end - start, end - start)  # Cria um circuito quântico para o bloco
            qc = apply_quantum_rule_based_on_classical_state(qc, block_state)  # Aplica a regra quântica baseada no estado clássico

            # Executar a simulação sem a medição para obter o statevector
            statevector = Statevector.from_instruction(qc)  # Obtém o vetor de estado do circuito
            plot_bloch_sphere(statevector.data)  # Plota a esfera de Bloch

            # Adicionar medição e executar a simulação para obter contagens
            qc.measure_all()  # Adiciona medições ao circuito

            aer_sim = AerSimulator()  # Inicializa o simulador de Aer
            transpiled_qc = transpile(qc, aer_sim)  # Transpila o circuito para otimização
            result = aer_sim.run(transpiled_qc, shots=1024, memory=True).result()  # Executa a simulação e obtém o resultado
            counts = result.get_counts()  # Obtém as contagens de cada estado
            block_results.append(counts)  # Adiciona as contagens ao resultado do bloco

            # Título e subtítulo detalhado
            title = f'Circuito Quântico {rule_number} - Bloco {block + 1} - Circuito {circuit_count}'
            subtitle = ('Este circuito quântico simula a regra {0}. '
                        'Primeiro, dividimos o estado inicial clássico em blocos menores para facilitar a simulação quântica. '
                        'Em seguida, configuramos os qubits (unidades de informação quântica) baseados em cada bloco desse estado inicial. '
                        'Depois, realizamos operações quânticas chamadas portas X (que são como "interruptores" que mudam o estado de 0 para 1 e vice-versa). '
                        'Essas portas são aplicadas aos qubits quando o estado inicial correspondente é 1. '
                        'Finalmente, medimos os qubits para ver o resultado dessas operações. '
                        'A medição nos mostra o estado final dos qubits após as operações.').format(rule_number)

            # Explicação Detalhada do Circuito
            explanation = (

                f"Título: {title}\n\n"
                f"Circuito Quântico {rule_number}: Indica que estamos simulando a regra {rule_number}.\n"
                f"Bloco {block + 1}: Significa que estamos no {block + 1}º bloco da divisão do estado inicial.\n"
                f"Circuito {circuit_count}:** Este é o {circuit_count}º circuito quântico na sequência de simulações.\n\n"
                f"Estado Inicial:\n"

                f"{subtitle}\n\n"
                f"- Componentes do Circuito\n"
                f"Qubits (`q[0]`, `q[1]`, `q[2]`), `q[n]`... :\n\n"
                f"As linhas horizontais representam os qubits. Cada linha é um qubit que pode estar nos estados 0 ou 1.\n\n"

                f"Células (`c[0]`, `c[1]`, `c[2]`), `c[n]`... :\n\n"
                f"As linhas verticais representam os bits clássicos. Cada linha é um bit clássico que pode estar nos estados 0 ou 1.\n\n"

                f"Operações:\n"
                f"- Porta X (NOT):\n"
                f"Representadas pelos quadrados cinza com um \"X\" dentro. Elas invertem o estado do qubit: de 0 para 1 ou de 1 para 0.\n"
                f"Neste circuito, a porta X é aplicada a `q[0]`, `q[1]`, `q[2]` e `q[n]`..., indicando que esses qubits estavam inicialmente no estado 1 no estado clássico inicial e agora estão invertidos.\n\n"

                f"Medições (`meas`):\n\n"
                f"Representadas pelas setas que apontam dos qubits para as linhas abaixo (marcadas como `c`). Medir um qubit é determinar seu estado final.\n"
                f"As medições são realizadas para `q[0]`, `q[1]`, `q[2]` e `q[n]`..., e os resultados são armazenados nos bits clássicos `c[0]`, `c[1]`, `c[2]` e ...`c[n]`.\n\n"

                f"Resultados:\n"
                f"- Blocos:\n"
                f"Para facilitar a simulação, o estado inicial clássico é dividido em blocos menores. Cada bloco contém um número fixo de células (ou qubits).\n"
                f"No exemplo, estamos observando o {block + 1}º bloco. Dividir o problema em blocos permite simular partes menores do problema sequencialmente, tornando a simulação mais gerenciável em um computador quântico.\n"
            )
            print(f"Circuito {circuit_count}:")
            print(qc)  # Imprime o circuito quântico
            print(f"Explicação Detalhada do Circuito {circuit_count}:")
            print(explanation)  # Imprime a explicação detalhada do circuito
            qc.draw('mpl', plot_barriers=True)  # Plota o circuito quântico
            plt.show()  # Mostra o gráfico
            plot_histogram(counts)  # Plota o histograma de contagens
            plt.show()  # Mostra o gráfico
            circuit_count += 1  # Incrementa o contador de circuitos
            block_results.append(counts)  # Adiciona as contagens ao resultado do bloco
            results.append(block_results)  # Adiciona os resultados do bloco aos resultados totais

            # Título e subtítulo detalhado
            print(f"Bloco {block + 1}: {block_state}")  # Imprime o estado do bloco
            print(f"Resultados do bloco {block + 1}: {block_results}")  # Imprime os resultados do bloco
            print()  # Imprime uma linha em branco para separar os blocos


            fig, ax = plt.subplots(figsize=(12, 6))
            ax.set_title(title)
            ax.text(0.5, -0.1, subtitle, ha='center', va='center', transform=ax.transAxes, wrap=True)
            qc.draw('mpl', ax=ax)
            plt.show()
            plot_histogram(counts)
            plt.show()
            circuit_count += 1  # Incrementa o contador de circuitos
            block_results.append(counts)  # Adiciona as contagens ao resultado do bloco
            results.append(block_results)  # Adiciona os resultados do bloco aos resultados totais


            circuit_count += 1  # Incrementa o contador de circuitos
        results.append(block_results)  # Adiciona os resultados do bloco aos resultados totais
    return results  # Retorna os resultados

# Função para converter resultados em DataFrame
def quantum_results_to_dataframe(quantum_results):
    data = []
    for step, block_results in enumerate(quantum_results):  # Itera sobre cada passo de tempo
        for block_index, result in enumerate(block_results):  # Itera sobre cada bloco
            for state, count in result.items():  # Itera sobre cada estado e sua contagem
                data.append({"Step": step, "Block": block_index, "State": state, "Count": count})  # Adiciona os dados ao DataFrame
    return pd.DataFrame(data)  # Retorna o DataFrame

# Função para juntar blocos e criar DataFrame total
def join_quantum_blocks(df_quantum, block_size, total_size):
    steps = df_quantum['Step'].max() + 1  # Obtém o número máximo de passos
    combined_data = []

    for step in range(steps):  # Itera sobre cada passo de tempo
        step_data = df_quantum[df_quantum['Step'] == step]  # Obtém os dados do passo atual
        combined_row = []
        for block in range(step_data['Block'].max() + 1):  # Itera sobre cada bloco
            block_data = step_data[step_data['Block'] == block]  # Obtém os dados do bloco atual
            if not block_data.empty:
                state_counts = block_data.groupby('State')['Count'].sum()  # Soma as contagens de cada estado
                most_frequent_state = state_counts.idxmax().replace(' ', '')  # Obtém o estado mais frequente e remove espaços
                combined_row.extend([int(bit) for bit in most_frequent_state])  # Adiciona o estado mais frequente à linha combinada
            else:
                combined_row.extend([0] * block_size)  # Adiciona zeros se o bloco estiver vazio
        combined_data.append([step] + combined_row[:total_size])  # Adiciona a linha combinada aos dados totais

    columns = ['Step'] + [f'Qubit_{i}' for i in range(total_size)]  # Cria os nomes das colunas
    return pd.DataFrame(combined_data, columns=columns)  # Retorna o DataFrame combinado

# Simulações clássicas
start_time = time.time()  # Marca o tempo de início
history_30_classical = simulate_classical(30)  # Simula a regra clássica 30
classical_30_time = time.time() - start_time  # Calcula o tempo de execução

start_time = time.time()
history_60_classical = simulate_classical(60)  # Simula a regra clássica 60
classical_60_time = time.time() - start_time

start_time = time.time()
history_90_classical = simulate_classical(90)  # Simula a regra clássica 90
classical_90_time = time.time() - start_time

df_30_classical = pd.DataFrame(history_30_classical)  # Converte a história dos estados para DataFrame
df_60_classical = pd.DataFrame(history_60_classical)
df_90_classical = pd.DataFrame(history_90_classical)

block_size = 7  # Define o tamanho do bloco
total_size = len(df_30_classical.columns)  # Obtém o tamanho total do estado

start_time = time.time()
counts_30_quantum = run_quantum_simulation_from_classical(df_30_classical, block_size, 30)  # Simula a regra quântica 30
quantum_30_time = time.time() - start_time

start_time = time.time()
counts_60_quantum = run_quantum_simulation_from_classical(df_60_classical, block_size, 60)  # Simula a regra quântica 60
quantum_60_time = time.time() - start_time

start_time = time.time()
counts_90_quantum = run_quantum_simulation_from_classical(df_90_classical, block_size, 90)  # Simula a regra quântica 90
quantum_90_time = time.time() - start_time

df_30_quantum = quantum_results_to_dataframe(counts_30_quantum)  # Converte os resultados quânticos para DataFrame
df_60_quantum = quantum_results_to_dataframe(counts_60_quantum)
df_90_quantum = quantum_results_to_dataframe(counts_90_quantum)

df_30_quantum_combined = join_quantum_blocks(df_30_quantum, block_size, total_size)  # Junta os blocos quânticos em um DataFrame total
df_60_quantum_combined = join_quantum_blocks(df_60_quantum, block_size, total_size)
df_90_quantum_combined = join_quantum_blocks(df_90_quantum, block_size, total_size)

df_30_classical.to_csv('df_30_classical.csv', index=False)  # Salva os DataFrames clássicos em arquivos CSV
df_60_classical.to_csv('df_60_classical.csv', index=False)
df_90_classical.to_csv('df_90_classical.csv', index=False)
df_30_quantum_combined.to_csv('df_30_quantum_combined.csv', index=False)  # Salva os DataFrames quânticos combinados em arquivos CSV
df_60_quantum_combined.to_csv('df_60_quantum_combined.csv', index=False)
df_90_quantum_combined.to_csv('df_90_quantum_combined.csv', index=False)

# Função para plotar a simulação clássica
def plot_classical_simulation(history, title):
    plt.figure(figsize=(10, 10))  # Define o tamanho da figura
    sns.heatmap(history, cmap='binary', cbar=True)  # Cria um mapa de calor
    plt.title(title)  # Define o título do gráfico
    plt.xlabel('Posição')  # Define o rótulo do eixo x
    plt.ylabel('Passo')  # Define o rótulo do eixo y
    plt.show()  # Mostra o gráfico

# Função para plotar a simulação quântica
def plot_quantum_simulation(df, title):
    steps = df['Step'].unique()  # Obtém os passos únicos
    df_heatmap = df.drop(columns='Step').set_index(steps)  # Prepara os dados para o mapa de calor
    plt.figure(figsize=(10, 10))
    sns.heatmap(df_heatmap, cmap='viridis', cbar=True)  # Cria um mapa de calor
    plt.title(title)
    plt.xlabel('Qubit')
    plt.ylabel('Passo')
    plt.show()

# Plota as simulações clássicas
plot_classical_simulation(df_30_classical.values, 'Simulação Clássica da Regra 30')
plot_classical_simulation(df_60_classical.values, 'Simulação Clássica da Regra 60')
plot_classical_simulation(df_90_classical.values, 'Simulação Clássica da Regra 90')

# Plota as simulações quânticas
plot_quantum_simulation(df_30_quantum_combined, 'Simulação Quântica da Regra 30')
plot_quantum_simulation(df_60_quantum_combined, 'Simulação Quântica da Regra 60')
plot_quantum_simulation(df_90_quantum_combined, 'Simulação Quântica da Regra 90')

# Função para calcular a entropia dos resultados quânticos
def calculate_entropy(df_quantum):
    entropies = []
    for step in df_quantum['Step'].unique():  # Itera sobre cada passo de tempo
        step_data = df_quantum[df_quantum['Step'] == step]  # Obtém os dados do passo atual
        counts = step_data.drop(columns=['Step']).values.flatten()  # Obtém as contagens
        entropies.append(entropy(counts[counts > 0]))  # Calcula a entropia
    return np.mean(entropies)  # Retorna a média das entropias

# Calcula a entropia das simulações quânticas
entropy_30_quantum = calculate_entropy(df_30_quantum_combined)
entropy_60_quantum = calculate_entropy(df_60_quantum_combined)
entropy_90_quantum = calculate_entropy(df_90_quantum_combined)

print(f"Entropia da Regra Quântica 30: {entropy_30_quantum}")
print(f"Entropia da Regra Quântica 60: {entropy_60_quantum}")
print(f"Entropia da Regra Quântica 90: {entropy_90_quantum}")

# Adiciona ruído aos resultados quânticos
noisy_counts_30_quantum = add_noise_to_quantum_results(counts_30_quantum)
noisy_counts_60_quantum = add_noise_to_quantum_results(counts_60_quantum)
noisy_counts_90_quantum = add_noise_to_quantum_results(counts_90_quantum)

# Converte os resultados com ruído para DataFrame
df_noisy_30_quantum = quantum_results_to_dataframe(noisy_counts_30_quantum)
df_noisy_60_quantum = quantum_results_to_dataframe(noisy_counts_60_quantum)
df_noisy_90_quantum = quantum_results_to_dataframe(noisy_counts_90_quantum)

# Junta os blocos com ruído em um DataFrame total
df_noisy_30_quantum_combined = join_quantum_blocks(df_noisy_30_quantum, block_size, total_size)
df_noisy_60_quantum_combined = join_quantum_blocks(df_noisy_60_quantum, block_size, total_size)
df_noisy_90_quantum_combined = join_quantum_blocks(df_noisy_90_quantum, block_size, total_size)

# Calcula a entropia dos resultados quânticos com ruído
noisy_entropy_30_quantum = calculate_entropy(df_noisy_30_quantum_combined)
noisy_entropy_60_quantum = calculate_entropy(df_noisy_60_quantum_combined)
noisy_entropy_90_quantum = calculate_entropy(df_noisy_90_quantum_combined)

print(f"Entropia da Regra Quântica 30 com Ruído: {noisy_entropy_30_quantum}")
print(f"Entropia da Regra Quântica 60 com Ruído: {noisy_entropy_60_quantum}")
print(f"Entropia da Regra Quântica 90 com Ruído: {noisy_entropy_90_quantum}")

# Função para plotar a entropia por passo
def plot_entropy_per_step(df_quantum, title):
    entropies = []
    steps = df_quantum['Step'].unique()  # Obtém os passos únicos
    for step in steps:  # Itera sobre cada passo de tempo
        step_data = df_quantum[df_quantum['Step'] == step]  # Obtém os dados do passo atual
        counts = step_data.drop(columns=['Step']).values.flatten()  # Obtém as contagens
        entropies.append(entropy(counts[counts > 0]))  # Calcula a entropia
    plt.figure(figsize=(10, 6))
    plt.plot(steps, entropies, marker='o')  # Plota a entropia por passo
    plt.title(title)
    plt.xlabel('Passo')
    plt.ylabel('Entropia')
    plt.grid(True)
    plt.show()

# Plota a entropia por passo para cada simulação
plot_entropy_per_step(df_30_quantum_combined, 'Entropia por Passo - Regra Quântica 30')
plot_entropy_per_step(df_60_quantum_combined, 'Entropia por Passo - Regra Quântica 60')
plot_entropy_per_step(df_90_quantum_combined, 'Entropia por Passo - Regra Quântica 90')

# Imprime o tempo de execução das simulações clássicas
print(f"Tempo de execução para Regra Clássica 30: {classical_30_time} segundos")
print(f"Tempo de execução para Regra Clássica 60: {classical_60_time} segundos")
print(f"Tempo de execução para Regra Clássica 90: {classical_90_time} segundos")

# Imprime o tempo de execução das simulações quânticas
print(f"Tempo de execução para Regra Quântica 30: {quantum_30_time} segundos")
print(f"Tempo de execução para Regra Quântica 60: {quantum_60_time} segundos")
print(f"Tempo de execução para Regra Quântica 90: {quantum_90_time} segundos")

# Verifica se os arquivos CSV existem antes de tentar carregá-los
def load_csv_if_exists(file_path):
    if os.path.exists(file_path):
        return pd.read_csv(file_path)
    else:
        print(f"Arquivo {file_path} não encontrado.")
        return None

# Carrega os DataFrames clássicos salvos
df_30_classical_loaded = load_csv_if_exists('df_30_classical.csv')
df_60_classical_loaded = load_csv_if_exists('df_60_classical.csv')
df_90_classical_loaded = load_csv_if_exists('df_90_classical.csv')

# Carrega os DataFrames quânticos combinados salvos
df_30_quantum_combined_loaded = load_csv_if_exists('df_30_quantum_combined.csv')
df_60_quantum_combined_loaded = load_csv_if_exists('df_60_quantum_combined.csv')
df_90_quantum_combined_loaded = load_csv_if_exists('df_90_quantum_combined.csv')

# Carrega os DataFrames de entropia salvos
df_entropy_30 = load_csv_if_exists('entropy_30.csv')
df_entropy_60 = load_csv_if_exists('entropy_60.csv')
df_entropy_90 = load_csv_if_exists('entropy_90.csv')

# Função para exibir os dados de um CSV
def display_csv_data(df, title):
    if df is not None:
        print(f"\n{title}")
        display(df)  # Exibe o DataFrame

# Exibe os DataFrames carregados
display_csv_data(df_30_classical_loaded, 'Tabela CSV - Simulação Clássica da Regra 30')
display_csv_data(df_60_classical_loaded, 'Tabela CSV - Simulação Clássica da Regra 60')
display_csv_data(df_90_classical_loaded, 'Tabela CSV - Simulação Clássica da Regra 90')
display_csv_data(df_30_quantum_combined_loaded, 'Tabela CSV - Simulação Quântica da Regra 30')
display_csv_data(df_60_quantum_combined_loaded, 'Tabela CSV - Simulação Quântica da Regra 60')
display_csv_data(df_90_quantum_combined_loaded, 'Tabela CSV - Simulação Quântica da Regra 90')
display_csv_data(df_entropy_30, 'Tabela CSV - Entropia por Passo - Regra Quântica 30')
display_csv_data(df_entropy_60, 'Tabela CSV - Entropia por Passo - Regra Quântica 60')
display_csv_data(df_entropy_90, 'Tabela CSV - Entropia por Passo - Regra Quântica 90')

# Cria um DataFrame com os tempos de execução e salva em CSV
execution_times = {
    'Simulação': ['Clássica 30', 'Clássica 60', 'Clássica 90', 'Quântica 30', 'Quântica 60', 'Quântica 90'],
    'Tempo (segundos)': [classical_30_time, classical_60_time, classical_90_time, quantum_30_time, quantum_60_time, quantum_90_time]
}
df_execution_times = pd.DataFrame(execution_times)
df_execution_times.to_csv('execution_times.csv', index=False)

# Cria um DataFrame com as entropias com ruído e salva em CSV
noisy_entropies = {
    'Regra': ['Quântica 30', 'Quântica 60', 'Quântica 90'],
    'Entropia': [noisy_entropy_30_quantum, noisy_entropy_60_quantum, noisy_entropy_90_quantum]
}
df_noisy_entropies = pd.DataFrame(noisy_entropies)
df_noisy_entropies.to_csv('noisy_entropies.csv', index=False)

# Carrega os DataFrames de tempos de execução e entropias com ruído
df_execution_times_loaded = load_csv_if_exists('execution_times.csv')
df_noisy_entropies_loaded = load_csv_if_exists('noisy_entropies.csv')

# Exibe os DataFrames carregados
display_csv_data(df_execution_times_loaded, 'Tabela CSV - Tempos de Execução')
display_csv_data(df_noisy_entropies_loaded, 'Tabela CSV - Entropia com Ruído')
